<a href="https://colab.research.google.com/github/Ian-Hardy/CSE_248_Adversarial_Recourse/blob/main/ca_data_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install carla-recourse
!pip install torchattacks
!pip install folktables

In [27]:
from carla import DataCatalog, MLModelCatalog
from carla.recourse_methods import GrowingSpheres
import numpy as np
import pandas as pd
import torch
import torchattacks
from torch.utils.data import TensorDataset, DataLoader
from folktables import ACSDataSource, ACSEmployment, ACSIncome

In [28]:
# load a catalog dataset
data_name = "adult"
dataset = DataCatalog(data_name)

# load artificial neural network from catalog
model = MLModelCatalog(dataset, "ann")

# get factuals from the data to generate counterfactual examples
factuals = dataset.raw.iloc[:10]

# load a recourse model and pass black box model
gs = GrowingSpheres(model)

# generate counterfactual examples
counterfactuals = gs.get_counterfactuals(factuals)

In [36]:
data_source = ACSDataSource(survey_year='2014', horizon='1-Year', survey='person')
ca_data = data_source.get_data(states=["CA"], download=True)
features, label, group = ACSIncome.df_to_numpy(ca_data)

<class 'pandas.core.frame.DataFrame'>


In [30]:
col_names = ['Age', 
            'Worker_Type', 
            'Education_Level', 
            'Marital_Status', 
            'Occupation_Code', 
            'State', 
            'Relationship', 
            'Hours_Per_Week', 
            'Sex', 
            'Race'
            ]
features_df = pd.DataFrame(features, columns=col_names)
features_df.head()

,Age,Worker_Type,Education_Level,Marital_Status,Occupation_Code,State,Relationship,Hours_Per_Week,Sex,Race
0,36.0,1.0,16.0,5.0,9610.0,22.0,12.0,35.0,1.0,1.0
1,47.0,1.0,20.0,3.0,5700.0,39.0,0.0,60.0,2.0,1.0
2,20.0,1.0,19.0,5.0,4840.0,6.0,2.0,40.0,1.0,1.0
3,31.0,1.0,19.0,1.0,5120.0,6.0,0.0,45.0,2.0,1.0
4,31.0,5.0,19.0,1.0,5550.0,6.0,1.0,40.0,1.0,1.0


In [31]:
col_name = ['Income_Greater_50k']
label_df = pd.DataFrame(label, columns=col_name)
label_df.head()

,Income_Greater_50k
0,False
1,False
2,False
3,True
4,False


In [32]:
X = pd.DataFrame(features_df).to_numpy()
y = pd.DataFrame(label_df).to_numpy()
y = y.ravel()

In [ ]:
for i, row in features_df.iterrows():
    print(i, row)

In [ ]:
from sklearn.neighbors import NearestNeighbors
number_of_diff_labels = []
nbrs = NearestNeighbors(n_neighbors=3).fit(features_df)
for i, row in counterfactuals.iterrows():
    knn = nbrs.kneighbors(row.values.reshape((1, -1)), 3, return_distance=False)[0]
    cf_label = row[y[i]]

    for idx in knn:
        neighbour = features_df.df.iloc[idx]
        # neighbour = neighbour.drop(label_df)
        neighbour = neighbour.values.reshape((1, -1))
        neighbour_label = np.argmax(ca_data.predict_proba(neighbour))

        number_of_diff_labels += np.abs(cf_label - neighbour_label)

# result = 1 - (1 / (N * y)) * number_of_diff_labels